In [2]:
import os
import json
import pandas as pd
import traceback

In [27]:
from dotenv import load_dotenv

In [28]:
load_dotenv()

True

In [29]:
KEY = os.getenv("OPENAI_API_KEY")

In [14]:
from langchain.chat_models import ChatOpenAI

In [15]:


llm = ChatOpenAI(openai_api_key="OPENAI_API_KEY",model_name="gpt-3.5-turbo",temperature=0.5)

In [32]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [35]:
TEMPLATE = """
Text : {text}
you are an exper MCQ maker. Given tha above text, it is your job to 
create a quize of {number} multiple choice questions for {subject} student in {tone} tone.
Make sure the questions are not repeated and check all the quesions to be confirming the text as weel.
Make sure format of you response like RESPONSE_JOSON below as use it as a guide.
Ensure to make {number} MCQS
###RESPONSE_JSON
{response_json}
"""

In [36]:
RESPONSE_JOSN = {
    "1":
    {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
            },
            "correct":"correct answer",
    },
    "2":
    {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
            },
            "correct":"correct answer",
    },
    "3":
    {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
            },
            "correct":"correct answer",
    },
    "4":
    {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
            },
            "correct":"correct answer",
    },
    "5":
    {
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
            },
            "correct":"correct answer",
    },

}

In [38]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template= TEMPLATE
    
)

In [40]:

quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt,output_key="quiz",verbose=True)

In [41]:


TEMPLATE2 = """
you are an english gramatian and writer. Given a multile choice Quiz for {subject} students.
You need to Evaluate the complexity of the questions and give a complete analysis of the quiz.
Only use at max 50 words for the complexity.
If the quiz is not at per with the cognitive and analytical abilities of the students,
update the quiz questions which newwds to be changed and change the tone such that it perfectly feets to the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English Witer of the above quiz
"""

In [42]:

quiz_evaluation_prompt= PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [46]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [52]:
generate_evaluation_chain = SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables = ["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],verbose=True
)


In [57]:
file_path = r"D:\GenAiProjects\mcqgen\data.txt"

In [61]:
with open(file_path,"r") as fp:
    TEXT = fp.read()

In [65]:
json.dumps(RESPONSE_JOSN)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [ ]:
with get_openai_callback() as cb:
    response = generate_evaluation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone"= TONE,
            "response_json": json.dumps(RESPONSE_JOSN)
        }
    )
    